<a href="https://colab.research.google.com/github/DLesmes/ML_Text_Classifier_Algorithms/blob/main/%5BLectura_16%5DSoluci%C3%B3n_Reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Construcción de un modelo markoviano de máxima entropía


In [2]:
!pip install conllu
!pip install stanza
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

     |████████████████████████████████| 286kB 13.0MB/s 
Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 578 (delta 4), reused 6 (delta 2), pack-reused 569
Receiving objects: 100% (578/578), 145.83 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (399/399), done.


### Entrenamiento del modelo - cálculo de conteos

Parta este modelo consideramos el cálculo de las probabilidades: 

$$P(t_i | w_i, t_{i-1}) =\frac{C(w_i, t_i, t_{i-1})}{C(w_i, t_{i-1})} $$

* `uniqueFeatureDict` $C(tag|word,prevtag) = C(w_i, t_i, t_{i-1})$
* `contextDict` $C(word,prevtag) = C(w_i, t_{i-1})$

En este caso cuando consideremos el primer elemento de una frase $w_0$, no existirá un elemento anterior $w_{-1}$ y por lo tanto, tampoco una etiqueta previa $t_{-1}$, podemos modelar este problema asumiendo que existe una etiqueta "None", para estos casos: 

$$P(t_0|w_0,t_{-1}) = P(t_0|w_0,\text{"None"})$$

In [3]:
from conllu import parse_incr

uniqueFeatureDict = {}
contextDict = {}

data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")

# Calculando conteos (pre-probabilidades)
for tokenlist in parse_incr(data_file):
  prevtag = "None"
  for token in tokenlist:
    tag = token['upos']
    word = token['form'].lower()
    largeKey = tag+'|'+word+','+prevtag
    if largeKey in uniqueFeatureDict.keys():
      uniqueFeatureDict[largeKey]+=1
    else:
      uniqueFeatureDict[largeKey]=1
    key = word+','+prevtag
    if key in contextDict.keys():
      contextDict[key]+=1
    else:
      contextDict[key]=1
    prevtag=tag

In [4]:
uniqueFeatureDict

{'DET|el,None': 2163,
 'NOUN|presidente,DET': 325,
 'ADP|del,NOUN': 3987,
 'NOUN|órgano,ADP': 3,
 'ADJ|regulador,NOUN': 4,
 'ADP|de,ADJ': 2329,
 'DET|las,ADP': 2367,
 'PROPN|telecomunicaciones,DET': 5,
 'PRON|se,PROPN': 415,
 'VERB|mostró,PRON': 47,
 'ADJ|partidario,VERB': 8,
 'VERB|completar,ADP': 7,
 'DET|esta,VERB': 104,
 'NOUN|liberalización,DET': 9,
 'ADP|de,NOUN': 16787,
 'NOUN|telecomunicaciones,DET': 7,
 'ADP|con,NOUN': 990,
 'DET|otras,ADP': 72,
 'NOUN|medidas,DET': 45,
 'PRON|que,NOUN': 2305,
 'VERB|incentiven,PRON': 1,
 'DET|la,VERB': 2291,
 'NOUN|competencia,DET': 37,
 'SCONJ|como,NOUN': 318,
 'NOUN|puede,SCONJ': 1,
 'NOUN|ser,NOUN': 2,
 'VERB|abrir,NOUN': 1,
 'DET|el,VERB': 2564,
 'NOUN|acceso,DET': 21,
 'ADP|a,NOUN': 1894,
 'DET|la,ADP': 9149,
 'NOUN|información,DET': 38,
 'DET|los,ADP': 4008,
 'NOUN|clientes,DET': 15,
 'PROPN|telefónica,ADP': 53,
 'ADP|a,PROPN': 256,
 'DET|otros,ADP': 98,
 'NOUN|operadores,DET': 7,
 'PUNCT|.,NOUN': 5650,
 'ADP|sobre,None': 26,
 'NOUN|ofe

In [5]:
contextDict

{'el,None': 2163,
 'presidente,DET': 326,
 'del,NOUN': 3987,
 'órgano,ADP': 3,
 'regulador,NOUN': 4,
 'de,ADJ': 2329,
 'las,ADP': 2367,
 'telecomunicaciones,DET': 12,
 'se,PROPN': 415,
 'mostró,PRON': 47,
 'partidario,VERB': 8,
 'completar,ADP': 7,
 'esta,VERB': 104,
 'liberalización,DET': 9,
 'de,NOUN': 16788,
 'con,NOUN': 990,
 'otras,ADP': 81,
 'medidas,DET': 45,
 'que,NOUN': 2755,
 'incentiven,PRON': 1,
 'la,VERB': 2382,
 'competencia,DET': 46,
 'como,NOUN': 326,
 'puede,SCONJ': 16,
 'ser,NOUN': 4,
 'abrir,NOUN': 1,
 'el,VERB': 2564,
 'acceso,DET': 21,
 'a,NOUN': 1900,
 'la,ADP': 9149,
 'información,DET': 39,
 'los,ADP': 4008,
 'clientes,DET': 15,
 'telefónica,ADP': 53,
 'a,PROPN': 256,
 'otros,ADP': 137,
 'operadores,DET': 7,
 '.,NOUN': 5650,
 'sobre,None': 26,
 'oferta,DET': 44,
 'interconexión,ADP': 5,
 'de,PROPN': 2321,
 'acaba,PRON': 11,
 'de,VERB': 1191,
 'aprobar,ADP': 5,
 'cmt,DET': 1,
 ',,PROPN': 7038,
 'vázquez,PUNCT': 1,
 'quintana,PROPN': 5,
 'dijo,PROPN': 53,
 'que,VER

### Entrenamiento del modelo - cálculo de probabilidades

$$P(t_i|w_i, t_{i-1}) = \frac{C(t_i, w_i, t_{i-1})}{C(w_i, t_{i-1})}$$

In [27]:
# uniqueFeatureDict = {}
# contextDict = {}
posteriorProbDict = {}

for key in uniqueFeatureDict.keys():
  if len(key.split('|'))==2:
    if key is posteriorProbDict.keys():
      posteriorProbDict[key] += uniqueFeatureDict[key]/contextDict[key.split('|')[1]]
    else:
      posteriorProbDict[key] = uniqueFeatureDict[key]/contextDict[key.split('|')[1]]


In [53]:
# Aquí verificamos que todas las probabilidades 
# por cada contexto 'word,prevtag' suman 1.0
sum_prob_uno = {}

for key in posteriorProbDict.keys():
  if len(key.split('|')) == 2:
    context = key.split('|')[1]
    if  context in sum_prob_uno.keys():
      sum_prob_uno[context] += posteriorProbDict[key]
    else:
      sum_prob_uno[context] = posteriorProbDict[key]

sum(sum_prob_uno.values())/len(sum_prob_uno)

1.0

### Distribución inicial de estados latentes

In [54]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
stateSet = {'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}
# enumeramos las categorias con numeros para asignar a 
# las columnas de la matriz de Viterbi
tagStateDict = {}
for i, state in enumerate(stateSet):
  tagStateDict[state] = i
tagStateDict

{'ADJ': 12,
 'ADP': 2,
 'ADV': 4,
 'AUX': 15,
 'CCONJ': 6,
 'DET': 1,
 'INTJ': 13,
 'NOUN': 8,
 'NUM': 5,
 'PART': 16,
 'PRON': 14,
 'PROPN': 3,
 'PUNCT': 9,
 'SCONJ': 11,
 'SYM': 0,
 'VERB': 10,
 '_': 7}

In [55]:
initTagStateProb = {} # \rho_i^{(0)}
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for tokenlist in parse_incr(data_file):
  count += 1
  tag = tokenlist[0]['upos']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'ADJ': 0.010136315973435861,
 'ADP': 0.1574274729115694,
 'ADV': 0.07577770010485844,
 'AUX': 0.022789234533379936,
 'CCONJ': 0.036980076896190144,
 'DET': 0.34799021321216356,
 'INTJ': 0.0020272631946871723,
 'NOUN': 0.025026214610276126,
 'NUM': 0.0068507514854945824,
 'PART': 0.002446696959105208,
 'PRON': 0.04173365955959455,
 'PROPN': 0.10506815798671792,
 'PUNCT': 0.09143656064313177,
 'SCONJ': 0.027123383432366307,
 'SYM': 0.0004893393918210416,
 'VERB': 0.04557846906675987,
 '_': 0.0011184900384480952}

### Construcción del algoritmo de Viterbi

Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array} 
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\ 
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n 
\end{array}
\right] 
\end{array}
$$

Donde las probabilidades de Viterbi en la primera columna (para una categoria $i$) están dadas por: 

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times P(i \vert p_1, \text{"None"})
$$

y para las siguientes columnas: 

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times P(j \vert p_t, i) \}
$$


In [56]:
import numpy as np 
import stanza
stanza.download('es')
nlp = stanza.Pipeline('es', processors='tokenize')
import nltk
nltk.download('punkt')
from nltk import word_tokenize

2021-04-12 04:23:17 INFO: Downloading default packages for language: es (Spanish)...
2021-04-12 04:25:13 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-04-12 04:25:13 WARNING: Language es package default expects mwt, which has been added
2021-04-12 04:25:13 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |

2021-04-12 04:25:13 INFO: Use device: cpu
2021-04-12 04:25:13 INFO: Loading: tokenize
2021-04-12 04:25:13 INFO: Loading: mwt
2021-04-12 04:25:13 INFO: Done loading processors!


In [58]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [74]:
def ViterbiMatrix(secuencia,
                  posteriorProbDict=posteriorProbDict,
                  initTagStateProb=initTagStateProb,
                  tagStateDict=tagStateDict
                  ):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))
    # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = key+'|'+seq[0].lower()+',None'
    if word_tag in posteriorProbDict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*posteriorProbDict[word_tag]

  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      for prevtag in tagStateDict.keys():
        word_tag = key+'|'+seq[col].lower()+','+prevtag
        if word_tag in posteriorProbDict.keys():
          #Buscamos el máximo
          possible_probs = []
          for key2 in tagStateDict.keys(): 
            tag_row2 = tagStateDict[key2]
            possible_probs.append(
                viterbiProb[tag_row2, col-1]*posteriorProbDict[word_tag])
          viterbiProb[tag_row, col] = max(possible_probs)
        
  return viterbiProb

matrix = ViterbiMatrix('gloria a Dios en el cielo')
matrix    

array([[0.00000000e+00, 8.52681929e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.52681929e-05, 1.39034526e-03, 0.00000000e+00,
        2.00209717e-02, 0.00000000e+00],
       [0.00000000e+00, 2.50262146e-02, 0.00000000e+00, 2.00209717e-02,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.26340964e-05, 2.00209717e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.50262146e-02, 2.55804579e-04, 5.00524292e-03, 0.00000000e+00,
        0.00000000e+00, 

In [75]:
def ViterbiTags(secuencia,
                  posteriorProbDict=posteriorProbDict,
                  initTagStateProb=initTagStateProb,
                  tagStateDict=tagStateDict
                  ):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))
    # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = key+'|'+seq[0].lower()+',None'
    if word_tag in posteriorProbDict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*posteriorProbDict[word_tag]

  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      for prevtag in tagStateDict.keys():
        word_tag = key+'|'+seq[col].lower()+','+prevtag
        if word_tag in posteriorProbDict.keys():
          #Buscamos el máximo
          possible_probs = []
          for key2 in tagStateDict.keys(): 
            tag_row2 = tagStateDict[key2]
            possible_probs.append(
                viterbiProb[tag_row2, col-1]*posteriorProbDict[word_tag])
          viterbiProb[tag_row, col] = max(possible_probs)
        
    # contruccion de secuencia de tags
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag))
      
  return res

ViterbiTags('gloria a Dios en el cielo')

[('gloria', 'NOUN'),
 ('a', 'ADP'),
 ('Dios', 'PROPN'),
 ('en', 'ADP'),
 ('el', 'DET'),
 ('cielo', 'NOUN')]

In [86]:
ViterbiTags('gracias a Dios, entiendo el ejercicio a la perfección')

[('gracias', 'NOUN'),
 ('a', 'ADP'),
 ('Dios', 'PROPN'),
 (',', 'PUNCT'),
 ('entiendo', 'VERB'),
 ('el', 'DET'),
 ('ejercicio', 'NOUN'),
 ('a', 'ADP'),
 ('la', 'DET'),
 ('perfección', 'NOUN')]

## ¿ Siguientes Pasos ? 

El modelo construido, aunque es la base de un MEMM, no explota todo el potencial del concepto  que estos modelos representan, en nuestro caso sencillo consideramos solo un **feature** para predecir la categoría gramatical: $<w_i, t_{i-1}>$. Es decir, las probabilidades de una cierta etiqueta $t_i$ dada una observación $<w_i, t_{i-1}>$ se calculan contando eventos donde se observe que $<w_i, t_{i-1}>$ sucede simultáneamente con $t_i$. 

La generalización de esto (donde puedo considerar multiples observaciones o **features**, y a partir de estos inferir la categoría gramatical) se hace construyendo las llamadas **feature-functions**, donde estas funciones toman valores de 0 o 1, cuando se cumplan las condiciones de la observación o feature en cuestion. En general podemos considerar una **feature-function** como : 

$$f_a(t, o) = f_a(\text{tag}, \text{observation}) = 
\begin{cases}
  1 , & \text{se cumple condición } a \\
  0, & \text{en caso contrario}
\end{cases}
$$

donde la condición $a$ es una relacion entre los valores que tome $\text{tag}$ y $\text{context}$, por ejemplo:

$$f_a(t, o) = f_a(\text{tag}, \text{observation}) = 
\begin{cases}
  1 , & (t_i, t_{i-1}) = \text{('VERB', 'ADJ')} \\
  0, & \text{en caso contrario}
\end{cases}
$$

Al considerar varias funciones, y por lo tanto varios features observables, consideramos una combinacion lineal de estos por medio de un coeficiente que multiplique a cada función: 

$$
\theta_1 f_1(t, o) + \theta_2 f_2(t, o) + \dots
$$

donde los coeficientes indicarán cuales features son más relevantes y por lo tanto pesan más para la decisión del resultado del modelo. De esta manera los coeficientes $\theta_j$ se vuelven parámetros del modelo que deben ser optimizados (esto puede realizarse con cualquier técnica de optimizacion como el Gradiente Descendente). Ahora, las probabilidades que pueden obtener usando un softmax sobre estas combinaciones lineales de features: 

$$
P = \prod_i \frac{\exp{\left(\sum_j \theta_j f_j(t_i, o)\right)}}{\sum_{t'}\exp{\left(\sum_j \theta_j f_j(t', o)\right)}}
$$

Así, lo que buscamos con el algoritmo de optimización es encontrar los parámetros $\theta_j$ que maximizan la probabilidad anterior. En NLTK encontramos la implementación completa de un clasificador de máxima entropia que no esta restringido a relaciones markovianas: https://www.nltk.org/_modules/nltk/classify/maxent.html

Un segmento resumido de la clase en python que implementa este clasificador en NLTK lo encuentras así: 

```
class MaxentClassifier(ClassifierI):

    def __init__(self, encoding, weights, logarithmic=True):
        self._encoding = encoding
        self._weights = weights
        self._logarithmic = logarithmic
        assert encoding.length() == len(weights)

    def labels(self):
        return self._encoding.labels()

    def set_weights(self, new_weights):
        self._weights = new_weights
        assert self._encoding.length() == len(new_weights)


    def weights(self):
        return self._weights

    def classify(self, featureset):
        return self.prob_classify(featureset).max()

    def prob_classify(self, featureset):
        ### ...

        # Normalize the dictionary to give a probability distribution
        return DictionaryProbDist(prob_dict, log=self._logarithmic, normalize=True)

    @classmethod
    def train(
        cls,
        train_toks,
        algorithm=None,
        trace=3,
        encoding=None,
        labels=None,
        gaussian_prior_sigma=0,
        **cutoffs
    ):
     ### ......
```

Donde te das cuenta de la forma que tienen las clases en NLTK que implementan clasificadores generales. Aquí vemos que la clase `MaxentClassifier` es una subclase de una más general `ClassifierI` la cual representa el proceso de clasificación general de categoría única (es decir, que a cada data-point le corresponda solo una categoria), también que esta clase depende de definir un `encoding`
 y unos pesos `weights` : 

```
class MaxentClassifier(ClassifierI):

    def __init__(self, encoding, weights, logarithmic=True):
```

los pesos corresponden a los parámetros $\theta_i$. Y el encoding es el que corresponde a las funciones $f_a(t, o)$ que dan como resultado valores binarios $1$ o $0$.

La documentación de NLTK te puede dar mas detalles de esta implementación: https://www.nltk.org/api/nltk.classify.html

Finalmente, un ejemplo completo de uso y mejora de un modelo de máxima entropía, lo puedes encontrar en este fork que guarde especialmente para el curso, para que lo tengas de referencia y puedas jugar y aprender con él: 

https://github.com/pachocamacho1990/nltk-maxent-pos-tagger

El cual fue desarrollado originalmente por Arne Neumann (https://github.com/arne-cl) basado en los fueatures propuestos por Ratnaparki en 1996 para la tarea de etiquetado por categorias gramaticales.
